In [89]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [90]:
from google.cloud import bigquery
from google.oauth2 import service_account

In [91]:
key_path = './service_account/gentle-keyword-423715-j0-03be08ad6412.json'

credentials = service_account.Credentials.from_service_account_file(
    key_path,
    scopes=["https://www.googleapis.com/auth/bigquery"]
)

In [92]:
from google.cloud import bigquery

client = bigquery.Client(
    credentials = credentials,
    project=credentials.project_id
)

In [93]:
# Create dataset reference
dataset_ref = client.dataset('google_analytics_sample', project='bigquery-public-data')
# Retrieve dataset from reference
dataset = client.get_dataset(dataset_ref)

In [94]:
# Try if this connected

query = """
SELECT *
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20160801`
LIMIT 5
"""

df = client.query(query).to_dataframe()
print(df.info())
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   visitorId             0 non-null      Int64 
 1   visitNumber           5 non-null      Int64 
 2   visitId               5 non-null      Int64 
 3   visitStartTime        5 non-null      Int64 
 4   date                  5 non-null      object
 5   totals                5 non-null      object
 6   trafficSource         5 non-null      object
 7   device                5 non-null      object
 8   geoNetwork            5 non-null      object
 9   customDimensions      5 non-null      object
 10  hits                  5 non-null      object
 11  fullVisitorId         5 non-null      object
 12  userId                0 non-null      object
 13  channelGrouping       5 non-null      object
 14  socialEngagementType  5 non-null      object
dtypes: Int64(4), object(11)
memory usage: 752.0+

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,channelGrouping,socialEngagementType
0,<NA>,1,1470046245,1470046245,20160801,"{'visits': 1, 'hits': 24, 'pageviews': 17, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Firefox', 'browserVersion': 'not ...","{'continent': 'Europe', 'subContinent': 'Weste...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 3, 'minut...",895954260133011192,None,Organic Search,Not Socially Engaged
1,<NA>,1,1470084717,1470084717,20160801,"{'visits': 1, 'hits': 24, 'pageviews': 18, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Internet Explorer', 'browserVersi...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 13, 'minu...",0288478011259077136,None,Direct,Not Socially Engaged
2,<NA>,3,1470078988,1470078988,20160801,"{'visits': 1, 'hits': 27, 'pageviews': 17, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Safari', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 12, 'minu...",6440789996634275026,None,Organic Search,Not Socially Engaged
3,<NA>,4,1470075581,1470075581,20160801,"{'visits': 1, 'hits': 27, 'pageviews': 19, 'ti...","{'referralPath': '/', 'campaign': '(not set)',...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 11, 'minu...",8520115029387302083,None,Referral,Not Socially Engaged
4,<NA>,30,1470099026,1470099026,20160801,"{'visits': 1, 'hits': 27, 'pageviews': 17, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 17, 'minu...",6792260745822342947,None,Organic Search,Not Socially Engaged


In [95]:
query = """
SELECT
  products.v2ProductCategory as category,
  date,
  SUM(totals.hits) as total_hits,
  SUM(totals.visits) as total_visits,
  SUM(totals.pageviews) as total_page_views,
  SUM(totals.newVisits) as total_first_visits,
  SUM(totals.timeOnScreen) as total_time_on_screen,
  AVG(totals.timeOnScreen) as avg_time_on_screen,
  SUM(totals.timeOnSite) as total_time_on_site,
  AVG(totals.timeOnSite) as avg_time_on_site,
  SUM(totals.screenviews) as total_screen_views,
  SUM(totals.transactions) as total_transactions,
  SUM(totals.transactionRevenue) as total_revenue,
  AVG(totals.transactionRevenue) as avg_revenue,
  SUM(totals.uniqueScreenviews) as total_unique_screen_views
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` h,
    UNNEST(h.hits) hits,
    UNNEST(hits.product) products
WHERE 
  (_TABLE_SUFFIX BETWEEN '20160801' AND '20170801')
  AND
    (
      hits.eCommerceAction.action_type != '0'
      AND
      hits.eCommerceAction.action_type != '3'
      AND
      hits.eCommerceAction.action_type != '4'
    )
  AND
  (geoNetwork.country = 'United States')
  AND
  (
  products.v2ProductCategory LIKE '%Accessories%'
  OR products.v2ProductCategory LIKE '%Electronics%'
  OR products.v2ProductCategory LIKE '%Office%'
  OR products.v2ProductCategory LIKE "%Men's%"
  OR products.v2ProductCategory LIKE "%Women's%"
  OR products.v2ProductCategory LIKE '%Shop by Brand%'
  )
GROUP BY category, date
"""

df_totals = client.query(query).to_dataframe()
print(df_totals.info())
df_totals.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9381 entries, 0 to 9380
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   category                   9381 non-null   object 
 1   date                       9381 non-null   object 
 2   total_hits                 9381 non-null   Int64  
 3   total_visits               9381 non-null   Int64  
 4   total_page_views           9381 non-null   Int64  
 5   total_first_visits         8593 non-null   Int64  
 6   total_time_on_screen       0 non-null      Int64  
 7   avg_time_on_screen         0 non-null      float64
 8   total_time_on_site         9379 non-null   Int64  
 9   avg_time_on_site           9379 non-null   float64
 10  total_screen_views         0 non-null      Int64  
 11  total_transactions         5815 non-null   Int64  
 12  total_revenue              5814 non-null   Int64  
 13  avg_revenue                5814 non-null   float

,category,date,total_hits,total_visits,total_page_views,total_first_visits,total_time_on_screen,avg_time_on_screen,total_time_on_site,avg_time_on_site,total_screen_views,total_transactions,total_revenue,avg_revenue,total_unique_screen_views
0,Electronics,20161027,2887,75,2218,45,<NA>,NaN,45102,601.360000,<NA>,70,17852500000,2.550357e+08,<NA>
1,Home/Apparel/Men's/Men's-Performance Wear/,20170618,1532,45,1013,3,<NA>,NaN,36649,814.422222,<NA>,25,1292710000,5.620478e+07,<NA>
2,Office,20170303,13733,167,10500,44,<NA>,NaN,254219,1522.269461,<NA>,66,11060140000,1.675779e+08,<NA>
3,Electronics,20161031,2655,66,2128,23,<NA>,NaN,105023,1591.257576,<NA>,41,18212570000,4.442090e+08,<NA>
4,Office,20170628,2255,55,1561,22,<NA>,NaN,74655,1357.363636,<NA>,35,53900190000,1.540005e+09,<NA>


In [96]:
df_totals = df_totals.dropna(axis=1, how='all').fillna(0)
df_totals['date'] = pd.to_datetime(df_totals['date'])

print(df_totals.info())
df_totals.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9381 entries, 0 to 9380
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   category            9381 non-null   object        
 1   date                9381 non-null   datetime64[ns]
 2   total_hits          9381 non-null   Int64         
 3   total_visits        9381 non-null   Int64         
 4   total_page_views    9381 non-null   Int64         
 5   total_first_visits  9381 non-null   Int64         
 6   total_time_on_site  9381 non-null   Int64         
 7   avg_time_on_site    9381 non-null   float64       
 8   total_transactions  9381 non-null   Int64         
 9   total_revenue       9381 non-null   Int64         
 10  avg_revenue         9381 non-null   float64       
dtypes: Int64(7), datetime64[ns](1), float64(2), object(1)
memory usage: 870.4+ KB
None


,category,date,total_hits,total_visits,total_page_views,total_first_visits,total_time_on_site,avg_time_on_site,total_transactions,total_revenue,avg_revenue
0,Electronics,2016-10-27,2887,75,2218,45,45102,601.360000,70,17852500000,2.550357e+08
1,Home/Apparel/Men's/Men's-Performance Wear/,2017-06-18,1532,45,1013,3,36649,814.422222,25,1292710000,5.620478e+07
2,Office,2017-03-03,13733,167,10500,44,254219,1522.269461,66,11060140000,1.675779e+08
3,Electronics,2016-10-31,2655,66,2128,23,105023,1591.257576,41,18212570000,4.442090e+08
4,Office,2017-06-28,2255,55,1561,22,74655,1357.363636,35,53900190000,1.540005e+09


In [97]:
def categorized(x):
    category = []
    categories = ['Accessories', 'Electronics', 'Office', 'Men\'s', 'Women\'s', 'Shop by Brand']
    for i in x:
        for cat in categories:
            if i.find(cat) != -1:
                category.append(cat)
    if len(category) == 0:
        return None
    return category[-1]

df_totals['category'] = df_totals['category'].str.split('/').apply(categorized)
df_totals = df_totals.groupby(['category', 'date']).mean().reset_index()
print(df_totals.info())
df_totals.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2170 entries, 0 to 2169
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   category            2170 non-null   object        
 1   date                2170 non-null   datetime64[ns]
 2   total_hits          2170 non-null   Float64       
 3   total_visits        2170 non-null   Float64       
 4   total_page_views    2170 non-null   Float64       
 5   total_first_visits  2170 non-null   Float64       
 6   total_time_on_site  2170 non-null   Float64       
 7   avg_time_on_site    2170 non-null   float64       
 8   total_transactions  2170 non-null   Float64       
 9   total_revenue       2170 non-null   Float64       
 10  avg_revenue         2170 non-null   float64       
dtypes: Float64(7), datetime64[ns](1), float64(2), object(1)
memory usage: 201.4+ KB
None


,category,date,total_hits,total_visits,total_page_views,total_first_visits,total_time_on_site,avg_time_on_site,total_transactions,total_revenue,avg_revenue
0,Accessories,2016-08-01,4566.333333,89.0,3027.666667,33.0,87104.333333,563.195850,20.666667,1459300000.0,2.353710e+07
1,Accessories,2016-08-02,5619.666667,94.666667,3713.0,40.333333,107344.333333,815.891196,10.0,629920000.0,2.422769e+07
2,Accessories,2016-08-03,6343.333333,113.666667,4077.0,58.666667,116221.333333,986.585195,0.0,0.0,0.000000e+00
3,Accessories,2016-08-04,5006.666667,95.666667,3549.333333,68.333333,141102.666667,805.315385,1.333333,122266666.666667,3.056667e+07
4,Accessories,2016-08-05,6448.0,110.0,4578.333333,70.0,139769.333333,799.491296,28.333333,3990420000.0,5.051165e+07


In [98]:
query = """
SELECT
  CASE 
    WHEN hits.eCommerceAction.action_type = '1' THEN 'Click through of product lists'
    WHEN hits.eCommerceAction.action_type = '2' THEN 'Product detail views'
    WHEN hits.eCommerceAction.action_type = '5' THEN 'Check out'
    WHEN hits.eCommerceAction.action_type = '6' THEN 'Completed purchase'
  END as action,
  products.v2ProductCategory as category,
  date,
  trafficSource.medium as medium,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` h,
    UNNEST(h.hits) hits,
    UNNEST(hits.product) products
WHERE 
  (_TABLE_SUFFIX BETWEEN '20160801' AND '20170801')
  AND
    (
      hits.eCommerceAction.action_type != '0'
      AND
      hits.eCommerceAction.action_type != '3'
      AND
      hits.eCommerceAction.action_type != '4'
    )
  AND
    (geoNetwork.country = 'United States')
  AND
  (
  products.v2ProductCategory LIKE '%Accessories%'
  OR products.v2ProductCategory LIKE '%Electronics%'
  OR products.v2ProductCategory LIKE '%Office%'
  OR products.v2ProductCategory LIKE "%Men's%"
  OR products.v2ProductCategory LIKE "%Women's%"
  OR products.v2ProductCategory LIKE '%Shop by Brand%'
  )
"""

df_hits_traffic = client.query(query).to_dataframe()
print(df_hits_traffic.info())
df_hits_traffic.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375065 entries, 0 to 375064
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   action    375065 non-null  object
 1   category  375065 non-null  object
 2   date      375065 non-null  object
 3   medium    375065 non-null  object
dtypes: object(4)
memory usage: 11.4+ MB
None


,action,category,date,medium
0,Click through of product lists,Home/Apparel/Men's/Men's-Outerwear/,20170205,(none)
1,Product detail views,Home/Apparel/Men's/Men's-Outerwear/,20170205,(none)
2,Product detail views,Home/Apparel/Men's/Men's-T-Shirts/,20170205,(none)
3,Click through of product lists,Home/Apparel/Men's/Men's-T-Shirts/,20170205,(none)
4,Click through of product lists,Home/Apparel/Men's/Men's-T-Shirts/,20170205,(none)


In [99]:
df_hits_traffic['date'] = pd.to_datetime(df_hits_traffic['date'])
df_hits_traffic['category'] = df_hits_traffic['category'].str.split('/').apply(categorized)

print(df_hits_traffic.info())
df_hits_traffic.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375065 entries, 0 to 375064
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   action    375065 non-null  object        
 1   category  375065 non-null  object        
 2   date      375065 non-null  datetime64[ns]
 3   medium    375065 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 11.4+ MB
None


,action,category,date,medium
0,Click through of product lists,Men's,2017-02-05,(none)
1,Product detail views,Men's,2017-02-05,(none)
2,Product detail views,Men's,2017-02-05,(none)
3,Click through of product lists,Men's,2017-02-05,(none)
4,Click through of product lists,Men's,2017-02-05,(none)


In [100]:
df_ohe = pd.get_dummies(df_hits_traffic[['medium', 'action']], dtype=int)
df_hits_traffic = pd.concat([df_hits_traffic.drop(['medium', 'action'], axis=1),
                            df_ohe], axis=1)

def modify_columns(array):
    temp = []
    for d in array:
        temp.append(d.replace('(', '').replace(')', '').replace(' ', '_'))
    return temp


df_hits_traffic.columns = modify_columns(np.array(df_hits_traffic.columns))

print(df_hits_traffic.info())
df_hits_traffic.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375065 entries, 0 to 375064
Data columns (total 13 columns):
 #   Column                                 Non-Null Count   Dtype         
---  ------                                 --------------   -----         
 0   category                               375065 non-null  object        
 1   date                                   375065 non-null  datetime64[ns]
 2   medium_none                            375065 non-null  int32         
 3   medium_not_set                         375065 non-null  int32         
 4   medium_affiliate                       375065 non-null  int32         
 5   medium_cpc                             375065 non-null  int32         
 6   medium_cpm                             375065 non-null  int32         
 7   medium_organic                         375065 non-null  int32         
 8   medium_referral                        375065 non-null  int32         
 9   action_Check_out                       375065 no

,category,date,medium_none,medium_not_set,medium_affiliate,medium_cpc,medium_cpm,medium_organic,medium_referral,action_Check_out,action_Click_through_of_product_lists,action_Completed_purchase,action_Product_detail_views
0,Men's,2017-02-05,1,0,0,0,0,0,0,0,1,0,0
1,Men's,2017-02-05,1,0,0,0,0,0,0,0,0,0,1
2,Men's,2017-02-05,1,0,0,0,0,0,0,0,0,0,1
3,Men's,2017-02-05,1,0,0,0,0,0,0,0,1,0,0
4,Men's,2017-02-05,1,0,0,0,0,0,0,0,1,0,0


In [101]:
df_hits_traffic = df_hits_traffic.groupby(['category', 'date']).sum().reset_index()

print(df_hits_traffic.info())
df_hits_traffic.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2170 entries, 0 to 2169
Data columns (total 13 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   category                               2170 non-null   object        
 1   date                                   2170 non-null   datetime64[ns]
 2   medium_none                            2170 non-null   int32         
 3   medium_not_set                         2170 non-null   int32         
 4   medium_affiliate                       2170 non-null   int32         
 5   medium_cpc                             2170 non-null   int32         
 6   medium_cpm                             2170 non-null   int32         
 7   medium_organic                         2170 non-null   int32         
 8   medium_referral                        2170 non-null   int32         
 9   action_Check_out                       2170 non-null   int32   

,category,date,medium_none,medium_not_set,medium_affiliate,medium_cpc,medium_cpm,medium_organic,medium_referral,action_Check_out,action_Click_through_of_product_lists,action_Completed_purchase,action_Product_detail_views
0,Accessories,2016-08-01,190,0,0,3,3,57,14,0,146,0,121
1,Accessories,2016-08-02,166,0,0,14,0,102,2,0,151,0,133
2,Accessories,2016-08-03,178,0,0,15,2,140,6,0,179,0,162
3,Accessories,2016-08-04,171,0,8,2,0,93,13,0,152,0,135
4,Accessories,2016-08-05,111,0,7,20,0,192,0,0,180,0,150


In [102]:
df = pd.merge(df_totals, df_hits_traffic, how='outer', on=['category', 'date'])
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2170 entries, 0 to 2169
Data columns (total 22 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   category                               2170 non-null   object        
 1   date                                   2170 non-null   datetime64[ns]
 2   total_hits                             2170 non-null   Float64       
 3   total_visits                           2170 non-null   Float64       
 4   total_page_views                       2170 non-null   Float64       
 5   total_first_visits                     2170 non-null   Float64       
 6   total_time_on_site                     2170 non-null   Float64       
 7   avg_time_on_site                       2170 non-null   float64       
 8   total_transactions                     2170 non-null   Float64       
 9   total_revenue                          2170 non-null   Float64 

,category,date,total_hits,total_visits,total_page_views,total_first_visits,total_time_on_site,avg_time_on_site,total_transactions,total_revenue,...,medium_not_set,medium_affiliate,medium_cpc,medium_cpm,medium_organic,medium_referral,action_Check_out,action_Click_through_of_product_lists,action_Completed_purchase,action_Product_detail_views
0,Accessories,2016-08-01,4566.333333,89.0,3027.666667,33.0,87104.333333,563.195850,20.666667,1459300000.0,...,0,0,3,3,57,14,0,146,0,121
1,Accessories,2016-08-02,5619.666667,94.666667,3713.0,40.333333,107344.333333,815.891196,10.0,629920000.0,...,0,0,14,0,102,2,0,151,0,133
2,Accessories,2016-08-03,6343.333333,113.666667,4077.0,58.666667,116221.333333,986.585195,0.0,0.0,...,0,0,15,2,140,6,0,179,0,162
3,Accessories,2016-08-04,5006.666667,95.666667,3549.333333,68.333333,141102.666667,805.315385,1.333333,122266666.666667,...,0,8,2,0,93,13,0,152,0,135
4,Accessories,2016-08-05,6448.0,110.0,4578.333333,70.0,139769.333333,799.491296,28.333333,3990420000.0,...,0,7,20,0,192,0,0,180,0,150


In [103]:
df.to_csv('data_modeling.csv', index=False)